### IMPORTS:

In [ ]:
# SELENIUM:
from selenium import webdriver                                    #driver
from selenium.webdriver.chrome.options import Options             #driver options
from webdriver_manager.chrome import ChromeDriverManager          #driver manager
from selenium.webdriver.common.by import By                       #for driver request
from selenium.webdriver.support.ui import WebDriverWait           #for waiting time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC  
#condiciones predefinidas que se pueden usar con el objeto WebDriverWait 
#para esperar a que se cumplan ciertas condiciones antes de continuar

# MONGODB:
import pymongo
from pymongo import MongoClient

# PERSONAL INFO:
from Passwords import password
from Passwords import email



### Conexión a MongoDB:

In [ ]:
cliente = MongoClient('localhost', 27017)
base_de_datos = cliente['scrap_link']
collection = base_de_datos['links_names']                     # colección donde guardamos los datos
collection.create_index([("enlace", pymongo.DESCENDING)]) 

### DRIVER CONECTION:

In [ ]:
chrome_options = Options() 
ChromeDriverManager().install()                                       #install
driver = webdriver.Chrome()                                           #initialization
driver.get("https://www.linkedin.com/login")

### LOGIN:

In [ ]:
def login(driver, email=None, password=None, cookie = None, timeout=10):
    if cookie is not None:
        return _login_with_cookie(driver, cookie)
  
    if not email or not password:
        email, password = __prompt_email_password()
  
    driver.get("https://www.linkedin.com/login")
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))
  
    email_elem = driver.find_element(By.ID,"username")
    email_elem.send_keys(email)
  
    password_elem = driver.find_element(By.ID,"password")
    password_elem.send_keys(password)
    password_elem.submit()
  
    element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CLASS_NAME, c.VERIFY_LOGIN_ID)))

In [ ]:
email = "email"
password = "password"
login(driver, email, password )

### SCRAP NAME-LINK, ANTIGUOS ALUMNOS IRONHACK:

In [ ]:
# Función para hacer scroll hacia abajo

def scroll_down():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Esperar hasta que aparezca un elemento específico antes de continuar
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "app-aware-link.link-without-visited-state"))
    )

In [ ]:
# Función para sacar link-name:

def scrap_links(url, collection):
    
    driver.get(url)

    try:
        while True:

            link_elements = driver.find_elements(By.CLASS_NAME, "app-aware-link.link-without-visited-state")

            for link_element in link_elements:
                link = link_element.get_attribute("href")
                name = link_element.text.strip()


                try:
                     collection.update_one(
                            {'enlace': link},            # Condición para buscar el documento existente por su enlace
                            {'$set': {'nombre': name}},  # Actualiza el nombre si el documento existe o crea uno nuevo
                            upsert=True)                 # Crea un nuevo documento si no se encuentra uno con la condición
                        
                except pymongo.errors.DuplicateKeyError as e:

                    continue

            scroll_down()

    except KeyboardInterrupt:
        pass

In [ ]:
url = 'https://www.linkedin.com/school/ironhack/people/'

In [ ]:
scrap_links(url, collection)